Link to article explainig code:
<url>https://cnvrg.io/pytorch-lstm/</url>

In [153]:
# imports
import torch as torch
import torch.nn as nn
from torch.autograd import Variable 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [154]:
# read processed data
df = pd.read_csv('../../dataframes/monthly_processed.csv', index_col='date', parse_dates=True)

In [155]:
# only use columns (x - y) as data
#df.drop(df.columns[np.r_[x, y:len(df.columns)]], axis=1, inplace=True)

# drop first x rows which contain metadata
#df.drop(df.index[0:x], axis=0, inplace=True)
X = df
X.loc[(X.index >= "2010-01-01") & (X.index < "2018-01-01")]

,fishoil_price_peru,fish_price_global,rapeseedoil_price_global,fishmeal_price_peru,peanutoil_price_global,soybeanoil_price_global,sunfloweroil_price_global
date,,,,,,,
2017-12-01,-0.240038,-0.128972,0.061253,-0.078029,0.052452,-0.003810,0.002150
2017-11-01,0.060447,-0.012739,0.065451,-0.063662,-0.005232,0.022328,0.011300
2017-10-01,-0.225434,0.110635,-0.070698,-0.004688,-0.014199,-0.006982,-0.006243
2017-09-01,-0.108276,0.008571,-0.027548,0.019360,0.015769,0.009989,0.021414
2017-08-01,0.038771,0.059369,0.002849,-0.014319,0.003662,-0.031662,-0.007937
...,...,...,...,...,...,...,...
2010-05-01,-0.050196,0.060328,-0.015622,-0.025545,0.000000,-0.000919,0.023347
2010-04-01,0.139669,0.025001,0.052665,-0.032722,-0.003848,0.049195,0.015267
2010-03-01,-0.280083,-0.078629,0.001044,-0.010348,-0.033304,0.009085,0.026697


In [156]:
# seperate predictors from output
X = df
y = df.iloc[:, 0:1]
y.head()

# split data into train and test test
X_train = X.loc[X.index < "2010-01-01"]
X_test = X.loc[X.index >= "2010-01-01"]
y_train = y.loc[X.index < "2010-01-01"]
y_test = y.loc[X.index >= "2010-01-01"]

In [157]:
X_train.values

array([[ 0.13541267, -0.01960847,  0.02613003, ..., -0.07626385,
         0.02081281,  0.01842427],
       [-0.24287086, -0.00795233, -0.00482891, ..., -0.02590223,
        -0.00804367, -0.06819632],
       [ 0.12772069, -0.04073883, -0.04340135, ...,  0.00326822,
        -0.03868459, -0.08494068],
       ...,
       [-0.0494833 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.50422805,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.3678824 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [158]:
# convert data to pytorch variables
X_train_tensors = Variable(torch.Tensor(X_train.values))
X_test_tensors = Variable(torch.Tensor(X_test.values))

y_train_tensors = Variable(torch.Tensor(y_train.values))
y_test_tensors = Variable(torch.Tensor(y_test.values)) 

print("Training Shape", X_train.shape, y_train.shape)
print("Testing Shape", X_test.shape, y_test.shape) 

Training Shape (300, 7) (300, 1)
Testing Shape (150, 7) (150, 1)


In [159]:
# reshape tensors to include timestamp
X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 

print("Training Shape", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape) 

Training Shape torch.Size([300, 1, 7]) torch.Size([300, 1])
Testing Shape torch.Size([150, 1, 7]) torch.Size([150, 1])


In [160]:
# definition of LSTM model
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes      # number of classes
        self.num_layers = num_layers        # number of layers
        self.input_size = input_size        # input size
        self.hidden_size = hidden_size      # hidden state

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) # lstm
        self.fc_1 =  nn.Linear(hidden_size, 128)        # fully connected 1
        self.fc = nn.Linear(128, num_classes)           # fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) # hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) # internal state
        # propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) # reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) # first Dense
        out = self.relu(out) # relu
        out = self.fc(out) # final Output
        return out

In [161]:
# variable definitions
num_epochs = 100            # 100 epochs
learning_rate = 0.001       # 0.001 lr

seq_length = 1              # sequence length

input_size = 7              # number of features
hidden_size = 2             # number of features in hidden state
num_layers = 1              # number of stacked lstm layers

num_classes = 1             # number of output classes 

In [ ]:
# transform train and test data to sequence of length seq_length
# TODO

In [ ]:
# create the lstm layer 
lstm = LSTM(num_classes, input_size, hidden_size, num_layers, seq_length) # lstm layer

In [ ]:
# define the loss function and weight updating method
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate) 

In [162]:
# train the model
for epoch in range(num_epochs):
  outputs = lstm.forward(X_train_tensors_final) # forward pass
  optimizer.zero_grad() # calculate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors)
 
  loss.backward() # calculates the loss of the loss function
 
  optimizer.step() # improve from loss, i.e backprop
  if epoch % 10 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.15744
Epoch: 10, loss: 0.05188
Epoch: 20, loss: 0.04882
Epoch: 30, loss: 0.04698
Epoch: 40, loss: 0.04449
Epoch: 50, loss: 0.04459
Epoch: 60, loss: 0.04398
Epoch: 70, loss: 0.04377
Epoch: 80, loss: 0.04343
Epoch: 90, loss: 0.04309


In [ ]:
# make predictions for the test set

# compute loss on test set

# plot test set predictions